In [ ]:
!pip install pymupdf azure-identity
!pip install openai==0.28
# NESTED ASYNCIO LOOP NEEDED TO RUN ASYNC IN A NOTEBOOK
import nest_asyncio

nest_asyncio.apply()
import openai
import re

# Récupération des variables depuis Colab userdata
from google.colab import userdata

# Configuration des paramètres pour Azure OpenAI
deployment_name="..."
model_name = "..."  # Nom du modèle
openai.api_key = "..."
openai.api_base = "..."
openai.api_version = '...'  # Version de l'API
openai.api_type = 'azure'  # Type de l'API


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.4
    Uninstalling openai-1.59.4:
      Successfully uninstalled openai-1.59.4


In [ ]:
import pandas as pd
df = pd.read_csv('RAG_QA.csv')
df.head()

,question,answer,expected_answer
0,Quel est le but de la politique de congés dans...,L'objectif de la politique de congés dans l'en...,La politique de congés de l'entreprise XXX vis...
1,Comment se fait le suivi des progrès liés au P...,Je ne dispose pas d'informations suffisantes s...,Le suivi des progrès réalisés dans le cadre du...
2,De quelle manière les employés peuvent-ils par...,Les employés peuvent contribuer positivement à...,Les employés sont encouragés à contribuer posi...
3,Quelles sont les différentes étapes du process...,Je ne connais pas la réponse. Cela n'est pas m...,Le département RH procède à une première sélec...
4,Quelle est la priorité de l'entreprise XXX con...,L'entreprise accorde la priorité au développem...,La formation et le développement des employés ...


In [ ]:
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



def get_embeddings(texts, deployment_name="text-embedding-ada-002"):
    """
    Obtenir les embeddings pour une liste de textes via Azure OpenAI.
    """
    response = openai.Embedding.create(
        input=texts,
        engine=deployment_name
    )
    embeddings = [item["embedding"] for item in response["data"]]
    return embeddings



In [ ]:
# Vérifier que les colonnes nécessaires sont présentes
required_columns = ["question", "answer", "expected_answer"]
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"Le fichier CSV doit contenir les colonnes suivantes : {required_columns}")

# Générer les embeddings pour chaque paire
expected_answers = df["answer"].tolist()
generated_answers = df["expected_answer"].tolist()

print("Génération des embeddings pour les réponses attendues...")
expected_embeddings = get_embeddings(expected_answers)

print("Génération des embeddings pour les réponses générées...")
generated_embeddings = get_embeddings(generated_answers)

# Calculer les similarités cosinus
print("Calcul des similarités cosinus...")
similarities = [
    cosine_similarity([expected], [generated])[0][0]
    for expected, generated in zip(expected_embeddings, generated_embeddings)
]

# Ajouter les scores de similarité au DataFrame
df["similarity_score"] = similarities

# Trier les lignes par le score de similarité décroissant
df_sorted = df.sort_values(by="similarity_score", ascending=False)

# Enregistrer les résultats dans un nouveau fichier CSV
output_file = "RAG_QA_withscore_sorted.csv"
df_sorted.to_csv(output_file, index=False)
print(f"Fichier sauvegardé avec les scores de similarité triés : {output_file}")
#faire la meme chose pour le finetuning et enregistrer le fichier FINETUNNING_QA_withscore.csv

Génération des embeddings pour les réponses attendues...
Génération des embeddings pour les réponses générées...
Calcul des similarités cosinus...
Fichier sauvegardé avec les scores de similarité triés : RAG_QA_withscore_sorted.csv


In [ ]:
score = pd.read_csv('RAG_QA_withscore_sorted.csv')

In [ ]:
score

,question,answer,expected_answer,similarity_score
0,Quelle est la durée du congé paternité pour le...,Les employés masculins ont droit à 4 semaines ...,Les employés masculins ont droit à un congé pa...,0.993537
1,Quelle est la mission principale de l'entrepri...,La mission de l'entreprise XXX est de fournir ...,La mission de l'entreprise XXX est de fournir ...,0.984390
2,Quelle est la stratégie de communication mise ...,L'entreprise XXX favorise une culture de commu...,L'entreprise XXX favorise une culture de commu...,0.984072
3,Que permet l'auto-évaluation dans le cadre de ...,L'auto-évaluation fait partie du processus d'é...,L'auto-évaluation dans le processus d'évaluati...,0.970440
4,Quels moyens sont à disposition des employés p...,Les employés peuvent signaler un comportement ...,Les employés peuvent signaler des comportement...,0.958609
5,Que comprend le programme d'intégration des no...,Le programme d'intégration des nouveaux employ...,Le programme d'intégration comprend une présen...,0.956550
6,Comment calcule-t-on les jours de congés pour ...,Les jours de congé annuel pour les employés à ...,Les employés à temps partiel bénéficient d'un ...,0.954515
7,Comment la politique de santé et de sécurité e...,L'entreprise XXX a une politique de santé et d...,La politique de santé et de sécurité de l'entr...,0.950050
8,Comment l'entreprise XXX protège-t-elle les em...,"Selon les politiques de l'entreprise, si un em...",L'entreprise XXX assure la protection de ses e...,0.931672
9,Quels types d'équipements de protection indivi...,La politique RH mentionne que l'entreprise fou...,L'entreprise XXX fournit à tous les employés l...,0.929772


In [ ]:
import pandas as pd

# Charger le fichier CSV
file_path = "RAG_QA_withscore.csv"  
df = pd.read_csv(file_path)

# Calculer les statistiques
moyenne_score = df['similarity_score'].mean()
min_score = df['similarity_score'].min()
max_score = df['similarity_score'].max()

# Afficher les résultats
print(f"Moyenne : {moyenne_score}")
print(f"Minimum : {min_score}")
print(f"Maximum : {max_score}")


Moyenne : 0.9023139576894593
Minimum : 0.7695098230413222
Maximum : 0.993536901243095


In [ ]:
import pandas as pd

# Charger le fichier CSV
file_path = "FINETUNNING_QA_withscore.csv"  
df = pd.read_csv(file_path)

# Calculer les statistiques
moyenne_score = df['similarity_score'].mean()
min_score = df['similarity_score'].min()
max_score = df['similarity_score'].max()

# Afficher les résultats
print(f"Moyenne : {moyenne_score}")
print(f"Minimum : {min_score}")
print(f"Maximum : {max_score}")


Moyenne : 0.9321422298748255
Minimum : 0.8949956489871811
Maximum : 0.9807533023315462


In [ ]:
import pandas as pd
score = pd.read_csv('FINETUNNING_QA_withscore.csv')

In [ ]:
score

,question,answer,fine-tuning-answer,similarity_score
0,Quel est le but de la politique de congés dans...,La politique de congés de l'entreprise XXX vis...,La politique de Congés dans notre entreprise v...,0.913805
1,Comment se fait le suivi des progrès liés au P...,Le suivi des progrès réalisés dans le cadre du...,Le suivi des progres réalisés dans le cadre du...,0.970348
2,De quelle manière les employés peuvent-ils par...,Les employés sont encouragés à contribuer posi...,Les employés sont encouragés à participer acti...,0.939401
3,Quelles sont les différentes étapes du process...,Le département RH procède à une première sélec...,Le département RH procède à une première sélec...,0.980753
4,Quelle est la priorité de l'entreprise XXX con...,La formation et le développement des employés ...,La formation et développement des employés est...,0.898069
5,Comment l'entreprise XXX protège-t-elle les em...,L'entreprise XXX assure la protection de ses e...,L'entreprise s'engage à protéger les employé·e...,0.950007
6,Quels types d'équipements de protection indivi...,L'entreprise XXX fournit à tous les employés l...,Des équipements de Protection Individuelle (EP...,0.967402
7,Quelle est la démarche recommandée pour prendr...,Les congés doivent être pris de manière raison...,Nous recommandons à tous les employés de prend...,0.910314
8,Que permet l'auto-évaluation dans le cadre de ...,L'auto-évaluation dans le processus d'évaluati...,L'autoévaluation permet à chaque employé de ré...,0.940667
9,Comment calcule-t-on les jours de congés pour ...,Les employés à temps partiel bénéficient d'un ...,Les employésà temps partiels bénéficient d'un ...,0.894996
